# Fine-tuning a Transformer Model for Multilingual Named Entity Recognition using the WikiNEuRal dataset (https://github.com/Babelscape/wikineural)

This guide will show you how to fine-tune 🤗 Transformers models for Named Entity Recognition. You will use the 🤗
Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

In [1]:
#! pip install datasets transformers seqeval

# Fine-tuning a model on a token classification task

Token classification refers to the task of classifying individual tokens in a sentence. One of the most common token
classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence,
such as a person, location, or organization. In this notebook, you will learn how to fine-tune a model on the [WikiNEuRal](https://github.com/Babelscape/wikineural) dataset to detect new entities.

## Loading the WikiNEuRal dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. 

The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [2]:
from datasets import load_dataset, load_metric, concatenate_datasets
import json
import pandas as pd
from datasets import Dataset

f = open('train3.json')
train_json = json.load(f)

f.close()



df = pd.DataFrame(train_json)

ds = Dataset.from_pandas(df)

ds[0]

{'tokens': ['Дисциплина',
  'посвящена',
  'изучению',
  'вопросов',
  ',',
  'связанных',
  'с',
  'целостным',
  'представлением',
  'о',
  'системах',
  'автоматизированного',
  'проектирования',
  ':',
  'видах',
  'их',
  'обеспечения',
  'и',
  'развитии',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'lang': 'ru'}

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [3]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

#labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

At this point, in order to obtain a multilingual model, we concatenate the training, validation and test sets of all languages. 

If you want to obtain a monolingual model, you have to use a single language for each split.

In [4]:
train_texts = pd.DataFrame(ds)

from sklearn.model_selection import train_test_split
train_texts, val_texts = train_test_split(pd.DataFrame(ds), test_size=.2)

train_texts[:10]


,tokens,ner_tags,lang
742,"[поиска, .]","[0, 0]",ru
1305,"[качества, .]","[0, 0]",ru
1063,"[и, глобальные, оптимальные, решения, .]","[0, 0, 0, 0, 0]",ru
710,"[:, Пустая, последовательность, отображаемых, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ru
1238,"[оценивание, .]","[0, 0]",ru
1180,"[главных, компонент, .]","[0, 0, 0]",ru
641,"[речевых, актов, .]","[0, 0, 0]",ru
303,"[собственного, приложения, ,, использующего, R...","[0, 0, 0, 0, 7, 0]",ru
1341,"[группового, выбора, .]","[0, 0, 0]",ru
947,"[интерполяция, .]","[0, 0]",ru


In [5]:
train_dataset = Dataset.from_pandas(train_texts)

val_dataset = Dataset.from_pandas(val_texts)

train_dataset[:5]

{'tokens': [['поиска', '.'],
  ['качества', '.'],
  ['и', 'глобальные', 'оптимальные', 'решения', '.'],
  [':',
   'Пустая',
   'последовательность',
   'отображаемых',
   'символов',
   'может',
   'быть',
   'значением',
   ':',
   '1',
   ')',
   'символьного',
   'типа',
   ',',
   '2',
   ')',
   'строкового',
   'типа',
   ',',
   '3',
   ')',
   'типа',
   '“',
   'имя',
   'экземпляра',
   '”.'],
  ['оценивание', '.']],
 'ner_tags': [[0, 0],
  [0, 0],
  [0, 0, 0, 0, 0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0]],
 'lang': ['ru', 'ru', 'ru', 'ru', 'ru'],
 '__index_level_0__': [742, 1305, 1063, 710, 1238]}

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
#model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

2023-01-24 19:59:20.682946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 19:59:20.790303: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 19:59:20.794379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 19:59:20.794392: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [7]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

Here the words "Ventana" and "condor" have been split in two subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [8]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [10]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
#test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [11]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [12]:
model_name = "Babelscape/wikineural-multilingual-ner"
args = TrainingArguments(
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [14]:
metric = load_metric("seqeval")

/tmp/ipykernel_32717/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [15]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [16]:

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/mary/Desktop/jupN/wikineural-multilingual-ner is already a clone of https://huggingface.co/oorschnai/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, __index_level_0__, lang. If tokens, ner_tags, __index_level_0__, lang are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/home/mary/Desktop/jupN/jupN/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1157
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 73
  Number of trainable parameters = 177269769
You're using a BertTokenizerFast tokenizer. Please note

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=73, training_loss=0.09521517035079329, metrics={'train_runtime': 236.5115, 'train_samples_per_second': 4.892, 'train_steps_per_second': 0.309, 'total_flos': 40821015284532.0, 'train_loss': 0.09521517035079329, 'epoch': 1.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [18]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, __index_level_0__, lang. If tokens, ner_tags, __index_level_0__, lang are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 290
  Batch size = 16


/home/mary/Desktop/jupN/jupN/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.10012322664260864,
 'eval_precision': 0.5714285714285714,
 'eval_recall': 0.43636363636363634,
 'eval_f1': 0.4948453608247423,
 'eval_accuracy': 0.9744588744588745,
 'eval_runtime': 15.8055,
 'eval_samples_per_second': 18.348,
 'eval_steps_per_second': 1.202,
 'epoch': 1.0}

In [19]:
trainer.push_to_hub("End of training")

Saving model checkpoint to wikineural-multilingual-ner
Configuration saved in wikineural-multilingual-ner/config.json
Model weights saved in wikineural-multilingual-ner/pytorch_model.bin
tokenizer config file saved in wikineural-multilingual-ner/tokenizer_config.json
Special tokens file saved in wikineural-multilingual-ner/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/676M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.31k/3.31k [00:00<?, ?B/s]

Upload file runs/Jan24_19-59-27_mary/1674579574.970193/events.out.tfevents.1674579574.mary.32717.1: 100%|#####…

Upload file runs/Jan24_19-59-27_mary/events.out.tfevents.1674579574.mary.32717.0: 100%|##########| 4.52k/4.52k…

Upload file runs/Jan24_19-59-27_mary/events.out.tfevents.1674579827.mary.32717.2: 100%|##########| 503/503 [00…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/oorschnai/wikineural-multilingual-ner
   b5cba49..4e92080  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.5714285714285714}, {'name': 'Recall', 'type': 'recall', 'value': 0.43636363636363634}, {'name': 'F1', 'type': 'f1', 'value': 0.4948453608247423}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9744588744588745}]}
To https://huggingface.co/oorschnai/wikineural-multilingual-ner
   4e92080..91838a9  main -> main



'https://huggingface.co/oorschnai/wikineural-multilingual-ner/commit/4e92080e4a13565f9dd5bb93e8ad1a3a220d0912'

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method: